# ✅ Skin Disease Classification from Scratch
This project avoids high-level libraries except where absolutely necessary. It is inspired by the Kaggle notebook [bhuvan2205/skin-disease-densenet-mobilenet](https://www.kaggle.com/code/bhuvan2205/skin-disease-densenet-mobilenet), but restructured to minimize library use, maximize transparency, and improve efficiency and accuracy.

In [ ]:
# 📦 Essential Imports
import os
import numpy as np
from PIL import Image
import random

In [ ]:
# 📂 Dataset Loader
class SkinDataset:
    def __init__(self, root_dir, img_size=(128,128)):
        self.samples = []
        self.img_size = img_size
        self.label_map = {name:i for i,name in enumerate(sorted(os.listdir(root_dir)))}
        for label_name, idx in self.label_map.items():
            folder = os.path.join(root_dir, label_name)
            for fname in os.listdir(folder):
                self.samples.append((os.path.join(folder, fname), idx))
        random.shuffle(self.samples)
    def __len__(self):
        return len(self.samples)
    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert('RGB').resize(self.img_size)
        arr = np.array(img, dtype=np.float32) / 255.0
        return arr.transpose(2,0,1), label

In [ ]:
# 🧱 CNN Layers
def relu(x): return np.maximum(0,x)

class ConvLayer:
    def __init__(self, in_c, out_c, k, stride=1, padding=1):
        self.W = np.random.randn(out_c, in_c, k, k) * np.sqrt(2/(in_c*k*k))
        self.b = np.zeros((out_c,1))
        self.stride, self.pad = stride, padding
    def forward(self, X):
        B,C,H,W = X.shape
        k = self.W.shape[2]
        out_h = (H+2*self.pad-k)//self.stride + 1
        out_w = (W+2*self.pad-k)//self.stride + 1
        X_pad = np.pad(X, ((0,0),(0,0),(self.pad,self.pad),(self.pad,self.pad)))
        out = np.zeros((B, self.W.shape[0], out_h, out_w))
        for i in range(out_h):
            for j in range(out_w):
                patch = X_pad[:,:,i*self.stride:i*self.stride+k, j*self.stride:j*self.stride+k]
                out[:,:,i,j] = np.tensordot(patch, self.W, axes=([1,2,3],[1,2,3])) + self.b[:,0]
        return out

In [ ]:
# 🧱 Max Pooling
class MaxPool:
    def __init__(self, k=2, stride=2):
        self.k, self.stride = k, stride
    def forward(self, X):
        B,C,H,W = X.shape
        out_h = H//self.k
        out_w = W//self.k
        out = np.zeros((B,C,out_h,out_w))
        for i in range(out_h):
            for j in range(out_w):
                patch = X[:,:,i*self.k:(i+1)*self.k,j*self.k:(j+1)*self.k]
                out[:,:,i,j] = patch.max(axis=(2,3))
        return out

In [ ]:
# 🧠 Model
class SimpleCNN:
    def __init__(self, num_classes):
        self.layers = [
            ConvLayer(3,16,3),
            ConvLayer(16,32,3),
            MaxPool(),
            ConvLayer(32,64,3),
            MaxPool()
        ]
        self.fc_W = np.random.randn(64*32*32, num_classes) * 0.01
        self.fc_b = np.zeros((num_classes,))
    def forward(self, X):
        out = X
        for l in self.layers:
            out = l.forward(out)
            if hasattr(l, 'W'):
                out = relu(out)
        B, C, H, W = out.shape
        flat = out.reshape(B, -1)
        logits = flat @ self.fc_W + self.fc_b
        return logits
    def predict_prob(self, X):
        logits = self.forward(X)
        ex = np.exp(logits - logits.max(axis=1, keepdims=True))
        return ex / ex.sum(axis=1, keepdims=True)

In [ ]:
# 🎯 Training & Evaluation
def cross_entropy_loss(probs, labels):
    B = labels.shape[0]
    return -np.mean(np.log(probs[np.arange(B), labels] + 1e-9))

def train_epoch(model, dataset, batch_size=16, lr=1e-3):
    total_loss = 0
    for i in range(0, len(dataset), batch_size):
        batch = [dataset[j] for j in range(i, min(i+batch_size, len(dataset)))]
        X = np.stack([b[0] for b in batch])
        Y = np.array([b[1] for b in batch])
        probs = model.predict_prob(X)
        loss = cross_entropy_loss(probs, Y)
        total_loss += loss * len(batch)
        # Add gradient code here if needed
    return total_loss / len(dataset)

def evaluate(model, dataset):
    correct, total = 0, 0
    for X, y in [dataset[i] for i in range(len(dataset))]:
        prob = model.predict_prob(X[None,...])[0]
        if prob.argmax() == y:
            correct += 1
        total += 1
    print(f"Accuracy: {correct/total*100:.2f}%")